### Imports & Env Setup

In [14]:
%reload_ext autoreload
%autoreload 2
import sys
import os
from dotenv import load_dotenv
load_dotenv()

import dspy
sys.path.append(os.path.abspath('../'))
from benchmarks import llama_mmlu_pro, leaderboard_mmlu_pro

### Configuration

In [35]:
NUM_THREADS = 16

FEW_SHOTS = 5

# See https://docs.litellm.ai/docs/providers/vllm for details
TASK_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1' , # or api_base ?
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)
PROMPT_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1', # or api_base ?
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)

dspy.configure(lm=TASK_MODEL)

# replace this with llama_mmlu_pro or whatever
benchmark = llama_mmlu_pro

# Without chain of thought:
# program = dspy.Predict(
#     benchmark.signature("")
# )

# With chain of thought:
program = dspy.ChainOfThought(
    benchmark.signature("You are a helpful assistant designed to help with multiple choice question.") # put your initial system prompt here, or leave blank
)

evaluate = dspy.Evaluate(
    devset=[],
    metric=benchmark.metric,
    num_threads=NUM_THREADS,
    display_progress=True,
    display_table=True,
    return_all_scores=True,
    return_outputs=True,
)

### Load dataset

In [36]:
trainset, valset, testset = benchmark.datasets(
    train_size=0.1,
    validation_size=0.2,
)

len(trainset), len(valset), len(testset)

(1197, 2156, 8626)

### Baseline Benchmark

In [37]:
print("BASE PROMPT:\n", program.predict.signature.instructions)

BASE PROMPT:
 You are a helpful assistant designed to help with multiple choice question.


In [39]:
eval_subset_size = 100
evaluate(
    program,
    devset=testset[:subset_size],
)

Average Metric: 71.00 / 99 (71.7%):  99%|████████████████████████████████████████▌| 99/100 [01:16<00:01,  1.58s/it]

2025/01/16 11:41:56 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'A (1/4) in. thick double leather belt is used on a cast steel pulley 50 in. in diameter which rotates at 1000 rpm and transmits 100 hp. Calculate the belt width using the following data: Coefficient of friction between cast-steel and leather = 0.40. Safe stress for belting = 300 psi Joint efficiency = 70 percent.', 'options': {'A': '7(1/2) in.', 'B': '7 in.', 'C': '9 in.', 'D': '6 in.', 'E': '5(1/2) in.', 'F': '9(1/2) in.', 'G': '10 in.', 'H': '8(1/2) in.', 'I': '8 in.', 'J': '11 in.'}, 'answer': 'I'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 71.00 / 99 (71.7%): 100%|████████████████████████████████████████| 100/100 [01:30<00:00,  1.11it/s]

2025/01/16 11:41:56 INFO dspy.evaluate.evaluate: Average Metric: 71.0 / 100 (71.0%)


,question,options,example_answer,reasoning,pred_answer,metric,answer
0,Describe the evolution of the reptilian excretory system to accoun...,{'A': 'The excretory system includes a secondary bladder for water...,J,The evolution of the reptilian excretory system from an aquatic to...,J,✔️ [True],NaN
1,A scientist used his car to transport a large quantity of highly f...,"{'A': 'No, because the doctor should have been more careful around...",D,To determine if the doctor will prevail in a claim against the sci...,D,✔️ [True],NaN
2,Which of the following could be used as a test for autocorrelation...,"{'A': 'The Dickey-Fuller test', 'B': 'The Jarque-Bera test', 'C': ...",G,To determine which of the following could be used as a test for au...,G,✔️ [True],NaN
3,Write the balanced cell reaction and calculate theemfat 298 K of t...,"{'A': '.25 V', 'B': '.114 V', 'C': '0.0157963 V', 'D': '.1298 V', ...",,"To solve this problem, we first need to write the balanced cell re...",B,,NaN
4,Assume a temperature of 300 K and find the wavelength of the photo...,"{'A': '2100.0', 'B': '2200.0', 'C': '1600.0', 'D': '1400.0', 'E': ...",G,To find the wavelength of the photon necessary to cause an electro...,J,,NaN
...,...,...,...,...,...,...,...
95,A pure lead bar 10 cm long is maintained with one end at T &=300 K...,"{'A': '2.56e-07', 'B': '6.40e-06', 'C': '6.40e-07', 'D': '5.12e-06...",H,"To find the thermoelectric power for lead, we first need to unders...",H,✔️ [True],NaN
96,Which of the following is another name for evading the issue?,"{'A': 'hasty generalization', 'B': 'slippery slope', 'C': '""you to...",G,"To answer this question, we need to understand what ""evading the i...",G,✔️ [True],NaN
97,A spherical charge distribution varies with the radius r by the eq...,"{'A': 'It increases as r approaches infinity.', 'B': 'It increases...",G,To determine how the electric field strength varies with distance ...,F,,NaN
98,Where in the balance sheet does each of the following belong? (A) ...,"{'A': ""(A) Liability section, (B) Asset side, (C) Owner's Equity s...",J,To determine where each of the given items belongs on the balance ...,J,✔️ [True],NaN


(71.0,
 [(Example({'question': 'Describe the evolution of the reptilian excretory system to account for the transition from an aquatic to a terrestrial habitat.', 'options': {'A': 'The excretory system includes a secondary bladder for water storage.', 'B': 'The reptilian excretory system has evolved to excrete more water', 'C': 'Reptiles have evolved to have a smooth, wet skin', 'D': 'Reptiles have evolved to excrete nitrogenous wastes primarily as ammonia to conserve water.', 'E': 'Reptiles have developed larger glomeruli to increase water retention.', 'F': 'The excretory system has adapted to increase salt excretion to facilitate life in marine environments.', 'G': 'Reptilian kidneys have evolved to produce ammonia directly for more efficient water use.', 'H': 'Reptiles excrete nitrogenous wastes as urea', 'I': 'The excretory system has evolved a complex series of ducts to recycle water back into the body.', 'J': 'The reptilian excretory system has evolved to conserve most of its wat

### Optimize Subset + Evaluation

In [17]:
subset_size = 20
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="light",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/16 11:20:46 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 5
valset size: 20

2025/01/16 11:20:46 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/16 11:20:46 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/16 11:20:46 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


 20%|███████████████▊                                                               | 4/20 [00:14<00:56,  3.50s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/5


 20%|███████████████▊                                                               | 4/20 [00:19<01:16,  4.75s/it]


...

2025/01/16 11:24:51 INFO dspy.evaluate.evaluate: Average Metric: 15 / 20 (75.0%)
2025/01/16 11:24:51 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 75.0 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 3'].
2025/01/16 11:24:51 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [70.0, 85.0, 80.0, 75.0, 80.0, 75.0]
2025/01/16 11:24:51 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 85.0
2025/01/16 11:24:51 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/16 11:24:51 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 6 / 7 =====



Average Metric: 17.00 / 20 (85.0%): 100%|██████████████████████████████████████████| 20/20 [00:17<00:00,  1.14it/s]

2025/01/16 11:25:08 INFO dspy.evaluate.evaluate: Average Metric: 17 / 20 (85.0%)
2025/01/16 11:25:08 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 85.0 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/01/16 11:25:08 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [70.0, 85.0, 80.0, 75.0, 80.0, 75.0, 85.0]
2025/01/16 11:25:08 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 85.0
2025/01/16 11:25:08 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/16 11:25:08 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 7 / 7 =====



Average Metric: 18.00 / 20 (90.0%): 100%|██████████████████████████████████████████| 20/20 [00:23<00:00,  1.17s/it]

2025/01/16 11:25:32 INFO dspy.evaluate.evaluate: Average Metric: 18 / 20 (90.0%)
2025/01/16 11:25:32 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far! Score: 90.0
2025/01/16 11:25:32 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 90.0 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 4'].
2025/01/16 11:25:32 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [70.0, 85.0, 80.0, 75.0, 80.0, 75.0, 85.0, 90.0]
2025/01/16 11:25:32 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 90.0
2025/01/16 11:25:32 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/16 11:25:32 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 90.0!


In [18]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

BEST PROMPT:
 You are a high-stakes test grader, responsible for evaluating the critical thinking and problem-solving skills of students in a high-pressure, timed environment. Given a complex, multiple-choice question that requires the application of technical vocabulary, critical thinking, analysis, and the ability to apply concepts to novel situations, along with a set of potential answer options, your task is to generate a step-by-step reasoning process for arriving at the correct answer, as well as identifying the correct answer itself. The question and options will be provided in the fields `question` and `options`, and you must produce detailed, logical reasoning in the field `reasoning` and the correct answer in the field `answer`. Your response will be evaluated not only on the correctness of the answer but also on the clarity, coherence, and logical soundness of the reasoning provided.


In [43]:
eval_subset_size = 200
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:eval_subset_size],
)

Average Metric: 2.00 / 2 (100.0%):   0%|▏                                          | 1/200 [00:00<00:19, 10.44it/s]

2025/01/16 12:03:59 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'A store wishes to make $12,000 profit on sales of $200,000. Find the markup percent on the selling price needed if expenses will be $56,000, markdowns $15,000, shortages $5,000, alteration costs $2,500, and cash discounts earned from vendors $ 4,500.', 'options': {'A': '33.33%', 'B': '39.09%', 'C': '25%', 'D': '37.5%', 'E': '42.5%', 'F': '40%', 'G': '35%', 'H': '28%', 'I': '45%', 'J': '30%'}, 'answer': 'B'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 5.00 / 6 (83.3%):   2%|█                                           | 5/200 [00:00<01:03,  3.07it/s]

2025/01/16 12:03:59 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Consider the initial value problem\n$$\n5 u^{\\prime \\prime}+2 u^{\\prime}+7 u=0, \\quad u(0)=2, \\quad u^{\\prime}(0)=1\n$$\nFind the smallest $T$ such that $|u(t)| \\leq 0.1$ for all $t>T$.', 'options': {'A': '18.6543', 'B': '8.9765', 'C': '11.1111', 'D': '10.1234', 'E': '14.5115', 'F': '22.2222', 'G': '9.8765', 'H': '16.7890', 'I': '12.3456', 'J': '20.2020'}, 'answer': ''}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 158.00 / 198 (79.8%): 100%|█████████████████████████████████████| 200/200 [00:01<00:00, 187.22it/s]

2025/01/16 12:04:00 INFO dspy.evaluate.evaluate: Average Metric: 158.0 / 200 (79.0%)


,question,options,example_answer,reasoning,pred_answer,metric,answer
0,Describe the evolution of the reptilian excretory system to accoun...,{'A': 'The excretory system includes a secondary bladder for water...,J,The transition from an aquatic to a terrestrial habitat imposed si...,J,✔️ [True],NaN
1,A scientist used his car to transport a large quantity of highly f...,"{'A': 'No, because the doctor should have been more careful around...",D,"To prevail in a claim based on strict liability, the doctor must s...",D,✔️ [True],NaN
2,Which of the following could be used as a test for autocorrelation...,"{'A': 'The Dickey-Fuller test', 'B': 'The Jarque-Bera test', 'C': ...",G,The question asks for a test that can be used to detect autocorrel...,G,✔️ [True],NaN
3,Write the balanced cell reaction and calculate theemfat 298 K of t...,"{'A': '.25 V', 'B': '.114 V', 'C': '0.0157963 V', 'D': '.1298 V', ...",,"To solve this problem, we first need to write the balanced cell re...",D,,NaN
4,Assume a temperature of 300 K and find the wavelength of the photo...,"{'A': '2100.0', 'B': '2200.0', 'C': '1600.0', 'D': '1400.0', 'E': ...",G,To find the wavelength of the photon necessary to cause an electro...,J,,NaN
...,...,...,...,...,...,...,...
195,Which statement is true?,{'A': 'All trapezoids are rectangles because they have at least on...,D,"To determine which statement is true, we need to evaluate each opt...",J,,NaN
196,"Select the best English interpretation of the given proposition, u...","{'A': 'All large apartments are bigger than some houses.', 'B': 'S...",E,The given proposition is (∃x)[(Ax • Lx) • (∃y)(Hy • Bxy)]. Breakin...,E,✔️ [True],NaN
197,"f(X) = [\pi(1 + X^2)]^-1- \infty < x < \infty. If Y = X^2, what is...",{'A': 'h(y) = [2 / {\\pi(1 + \\sqrt{y})}] for y > 0 and = 0 otherw...,G,"To find the density function of Y, given that Y = X^2, we first ne...",G,✔️ [True],NaN
198,Two thin convex lenses of focal lengths f_1 and f_2 are separated ...,"{'A': '[(3f_2) / 2]', 'B': '(f_1 + f_2) / 2', 'C': '(2f_2) / 3', '...",A,The focal length of the combination of two thin convex lenses can ...,A,✔️ [True],NaN


## Medium Optimization

In [44]:
subset_size = 500
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="medium",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/16 12:04:38 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 19
valset size: 300

2025/01/16 12:04:38 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/16 12:04:38 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/16 12:04:38 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


  1%|▊                                                                             | 5/500 [00:24<40:21,  4.89s/it]


Bootstrapped 4 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.
Bootstrapping set 4/19


  1%|▍                                                                             | 3/500 [00:11<31:41,  3.83s/it]


...

2025/01/16 13:17:51 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Consider the initial value problem\n$$\ny^{\\prime \\prime}+\\gamma y^{\\prime}+y=k \\delta(t-1), \\quad y(0)=0, \\quad y^{\\prime}(0)=0\n$$\nwhere $k$ is the magnitude of an impulse at $t=1$ and $\\gamma$ is the damping coefficient (or resistance).\nLet $\\gamma=\\frac{1}{2}$. Find the value of $k$ for which the response has a peak value of 2 ; call this value $k_1$.', 'options': {'A': '3.1415', 'B': '3.9022', 'C': ' 2.8108', 'D': '2.0000', 'E': '3.5672', 'F': '2.3456', 'G': '4.0000', 'H': '2.7182', 'I': '1.7890', 'J': '1.6180'}, 'answer': ''}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 223.00 / 298 (74.8%): 100%|█████████████████████████████████████▊| 299/300 [03:56<00:07,  7.34s/it]

2025/01/16 13:18:50 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'An aluminum calorimeter of mass 50 g contains 95 g of a mixture of water and ice at 0°C. When 100 g of aluminum which has been heated in a steam jacket is dropped into the mixture, the temperature rises to 5°C. Find the mass of ice originally present if the specific heat capacity of aluminum is 0.22 cal/g\\bulletCdeg.', 'options': {'A': '13.0 g', 'B': '19.50 g', 'C': '22.0 g', 'D': '17.5 g', 'E': '25.0 g', 'F': '16.0 g', 'G': '20.0 g', 'H': '18.0 g', 'I': '21.0 g', 'J': '15.0 g'}, 'answer': 'G'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 223.00 / 298 (74.8%): 100%|██████████████████████████████████████| 300/300 [04:10<00:00,  1.20it/s]

2025/01/16 13:18:50 INFO dspy.evaluate.evaluate: Average Metric: 223.0 / 300 (74.3%)
2025/01/16 13:18:50 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [72.67, 74.33, 76.33, 74.33]
2025/01/16 13:18:50 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 76.33
2025/01/16 13:18:50 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/16 13:18:50 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/16 13:18:50 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 76.33!


In [ ]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

In [45]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

BEST PROMPT:
 You are a helpful assistant designed to help with multiple-choice questions. Given a question and a set of options, analyze the question carefully, considering the context and the principles involved. Then, evaluate each option in relation to the question, selecting the most appropriate answer based on your analysis. Provide a step-by-step reasoning for your choice, explaining why you selected a particular option and why the others are incorrect. Ensure your reasoning is clear, concise, and directly addresses the question being asked.


In [55]:
print("BEST EXAMPLES:\n", optimized_program.predict.demos)

BEST EXAMPLES:
 [Example({'augmented': True, 'question': 'Howarethe type of muscle tissues differentiated?', 'options': {'A': 'Muscle tissues are differentiated by the presence or absence of striations under a microscope', 'B': 'Muscle tissues are differentiated by the types of hormones they respond to', 'C': 'Muscle tissues are differentiated by their color under standard lighting conditions', 'D': 'Muscle tissues are differentiated by their size and shape', 'E': 'Muscle tissues are differentiated by their ability to produce electrical impulses', 'F': 'Muscle tissues are differentiated by their ability to heal and regenerate', 'G': 'Muscle tissues are differentiated by the number of nuclei per cell', 'H': 'Muscle tissues are differentiated by the speed of contraction and relaxation', 'I': 'Muscle tissues are differentiated by their location in the body', 'J': 'Skeletal muscle is responsible for most voluntary movements, smooth muscle is involved in most involuntary movements of intern

In [46]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:subset_size],
    display_table=False,
)

Average Metric: 386.00 / 500 (77.2%): 100%|██████████████████████████████████████| 500/500 [31:43<00:00,  3.81s/it]

2025/01/16 14:00:46 INFO dspy.evaluate.evaluate: Average Metric: 386 / 500 (77.2%)


In [51]:
eval_medium_subset_size = 300
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:eval_medium_subset_size],
)

Average Metric: 236.00 /


2025/01/16 15:20:02 INFO dspy.evaluate.evaluate: Average Metric: 236 / 300 (78.7%)


,question,options,example_answer,reasoning,pred_answer,metric
0,Describe the evolution of the reptilian excretory system to accoun...,{'A': 'The excretory system includes a secondary bladder for water...,J,The evolution of the reptilian excretory system from an aquatic to...,J,✔️ [True]
1,A scientist used his car to transport a large quantity of highly f...,"{'A': 'No, because the doctor should have been more careful around...",D,To determine if the doctor will prevail in a claim against the sci...,D,✔️ [True]
2,Which of the following could be used as a test for autocorrelation...,"{'A': 'The Dickey-Fuller test', 'B': 'The Jarque-Bera test', 'C': ...",G,The Breusch-Godfrey test is a statistical test used to detect auto...,G,✔️ [True]
3,Write the balanced cell reaction and calculate theemfat 298 K of t...,"{'A': '.25 V', 'B': '.114 V', 'C': '0.0157963 V', 'D': '.1298 V', ...",,"To solve this problem, we need to write the balanced cell reaction...",D,
4,Assume a temperature of 300 K and find the wavelength of the photo...,"{'A': '2100.0', 'B': '2200.0', 'C': '1600.0', 'D': '1400.0', 'E': ...",G,To find the wavelength of the photon necessary to cause an electro...,J,
...,...,...,...,...,...,...
295,We were first able to accurately measure the diameter of Pluto from:,"{'A': ""Lunar-based observations made during NASA's Apollo missions...",H,The diameter of Pluto was first accurately measured through observ...,D,
296,Which of the following is a clustering algorithm in machine learning?,"{'A': 'Linear Regression', 'B': 'CART', 'C': 'Logistic Regression'...",D,Clustering algorithms in machine learning are used to group simila...,D,✔️ [True]
297,"In a population in Denmark, the relative fitness of the allele for...","{'A': '7.82 × 10^-5', 'B': '3.14 × 10^-5', 'C': '1.19 × 10^-4', 'D...",H,"To find the mutation rate, we first need to understand the relatio...",D,
298,Miss Jones has been concerned about her health lately. She has not...,"{'A': 'herpes', 'B': 'trichomoniasis', 'C': 'pubic lice', 'D': 'sy...",C,Given Miss Jones' symptoms of itching and skin irritation around h...,C,✔️ [True]


## Heavy Optimization

In [ ]:
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="heavy",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset,
    valset=valset,
)

In [ ]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

In [ ]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset,
    display_table=False,
)